# ตัวจำแนกภาพแบบง่าย

โน้ตบุ๊กนี้จะแสดงวิธีการจำแนกภาพโดยใช้เครือข่ายประสาทที่ผ่านการฝึกมาแล้ว

**สิ่งที่คุณจะได้เรียนรู้:**
- วิธีโหลดและใช้งานโมเดลที่ผ่านการฝึกมาแล้ว
- การเตรียมภาพก่อนประมวลผล
- การทำนายผลบนภาพ
- การทำความเข้าใจคะแนนความมั่นใจ

**กรณีการใช้งาน:** ระบุวัตถุในภาพ (เช่น "แมว", "สุนัข", "รถยนต์" เป็นต้น)

---


## ขั้นตอนที่ 1: นำเข้าไลบรารีที่จำเป็น

มานำเครื่องมือที่เราต้องใช้กันเถอะ ไม่ต้องกังวลถ้าคุณยังไม่เข้าใจทั้งหมดในตอนนี้!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## ขั้นตอนที่ 2: โหลดโมเดลที่ผ่านการฝึกมาแล้ว

เราจะใช้ **MobileNetV2** ซึ่งเป็นเครือข่ายประสาทที่ผ่านการฝึกด้วยภาพนับล้านมาแล้ว

สิ่งนี้เรียกว่า **Transfer Learning** - การใช้โมเดลที่มีคนอื่นฝึกไว้แล้ว!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## ขั้นตอนที่ 3: ฟังก์ชันช่วยเหลือ

มาสร้างฟังก์ชันสำหรับโหลดและเตรียมรูปภาพให้พร้อมสำหรับโมเดลของเรากันเถอะ


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## ขั้นตอนที่ 4: ทดสอบกับภาพตัวอย่าง

ลองมาทำการจำแนกภาพบางภาพจากอินเทอร์เน็ตกัน!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### จำแนกแต่ละภาพ


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## ขั้นตอนที่ 5: ลองใช้ภาพของคุณเอง!

เปลี่ยน URL ด้านล่างด้วย URL ของภาพใดก็ได้ที่คุณต้องการจำแนกประเภท.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 เกิดอะไรขึ้น?

1. **เราโหลดโมเดลที่ถูกฝึกมาแล้ว** - MobileNetV2 ถูกฝึกด้วยภาพนับล้าน
2. **เราเตรียมภาพให้พร้อมใช้งาน** - ปรับขนาดและจัดรูปแบบให้เหมาะกับโมเดล
3. **โมเดลทำการทำนาย** - ให้ผลลัพธ์เป็นความน่าจะเป็นสำหรับวัตถุ 1000 ประเภท
4. **เราแปลงผลลัพธ์** - เปลี่ยนตัวเลขให้เป็นชื่อที่มนุษย์เข้าใจได้

### ทำความเข้าใจคะแนนความมั่นใจ

- **90-100%**: มั่นใจมาก (เกือบจะถูกต้องแน่นอน)
- **70-90%**: มั่นใจ (น่าจะถูกต้อง)
- **50-70%**: มั่นใจระดับหนึ่ง (อาจจะถูกต้อง)
- **ต่ำกว่า 50%**: ไม่ค่อยมั่นใจ (ไม่แน่ใจ)

### ทำไมการทำนายอาจผิดพลาด?

- **มุมหรือแสงที่ผิดปกติ** - โมเดลถูกฝึกด้วยภาพทั่วไป
- **มีวัตถุหลายอย่างในภาพ** - โมเดลคาดหวังว่าจะมีวัตถุหลักเพียงหนึ่งเดียว
- **วัตถุที่พบได้ยาก** - โมเดลรู้จักเพียง 1000 ประเภท
- **ภาพคุณภาพต่ำ** - ภาพเบลอหรือมีพิกเซลต่ำทำให้วิเคราะห์ยาก

---


## 🚀 ขั้นตอนถัดไป

1. **ลองใช้ภาพที่แตกต่างกัน:**
   - ค้นหาภาพจาก [Unsplash](https://unsplash.com)
   - คลิกขวา → "คัดลอกที่อยู่ภาพ" เพื่อรับ URL

2. **ทดลอง:**
   - จะเกิดอะไรขึ้นกับงานศิลปะเชิงนามธรรม?
   - สามารถจดจำวัตถุจากมุมมองที่แตกต่างกันได้หรือไม่?
   - มันจัดการกับวัตถุหลายชิ้นอย่างไร?

3. **เรียนรู้เพิ่มเติม:**
   - สำรวจ [บทเรียนเกี่ยวกับ Computer Vision](../lessons/4-ComputerVision/README.md)
   - เรียนรู้วิธีฝึกฝนตัวจำแนกภาพของคุณเอง
   - ทำความเข้าใจว่า CNNs (Convolutional Neural Networks) ทำงานอย่างไร

---

## 🎉 ขอแสดงความยินดี!

คุณเพิ่งสร้างตัวจำแนกภาพโดยใช้เครือข่ายประสาทเทียมที่ล้ำสมัย!

เทคนิคเดียวกันนี้ถูกนำไปใช้ใน:
- Google Photos (จัดระเบียบรูปภาพของคุณ)
- รถยนต์ไร้คนขับ (จดจำวัตถุ)
- การวินิจฉัยทางการแพทย์ (วิเคราะห์ภาพเอ็กซ์เรย์)
- การควบคุมคุณภาพ (ตรวจจับข้อบกพร่อง)

อย่าหยุดสำรวจและเรียนรู้ต่อไป! 🚀



---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามให้การแปลมีความถูกต้อง แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาต้นทางควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ แนะนำให้ใช้บริการแปลภาษามนุษย์ที่เป็นมืออาชีพ เราจะไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดที่เกิดจากการใช้การแปลนี้
